## 챗봇 학습 데이터 엑셀 파일 및 DB 연동

In [14]:
# 필요한 모듈 임포트
import pymysql
import openpyxl

In [15]:
# 학습 데이터 초기화
def all_clear_train_data(db):
    # 기존 학습 데이터 삭제
    sql='''
        delete from chatbot_train_data
    '''
    with db.cursor() as cursor:
        cursor.execute(sql)
        
    # auto increment 초기화
    sql='''
        ALTER TABLE chatbot_train_data AUTO_INCREMENT=1
    '''
    with db.cursor() as cursor:
        cursor.execute(sql)

### 학습 데이터 초기화
데이터를 모두 지우고 새로 입력하는 구조 <br>
delect를 통해 삭제하고 auto increment 1로 초기화하는 함수

In [16]:
# db에 데이터 저장
def insert_data(db, xls_row):
    intent, ner, query, answer, answer_img_url = xls_row
    
    sql='''
        INSERT chatbot_train_data(intent, ner, query, answer, answer_img_url)
        values(
            '%s', '%s', '%s', '%s', '%s'
        )
    ''' % (intent.value, ner.value, query.value, answer.value, answer_img_url.value)
    
    # 엑셀에서 불러온 cell에 데이터가 없는 경우 null로 치환
    sql = sql.replace("'None'", "null")
    
    with db.cursor() as cursor:
        cursor.execute(sql)
        print('{} 저장'.format(query.value))
        db.commit()

In [17]:
# 학습 엑셀 파일 선언
train_file = './train_data.xlsx'

In [18]:
db = None
try:
    db = pymysql.connect(
        host="127.0.0.1",
        user="homestead",
        passwd="12test34",
        db="homestead",
        charset='utf8'
    )

    # 기존 학습 데이터 초기화
    all_clear_train_data(db)

    # 학습 엑셀 파일 불러오기
    wb = openpyxl.load_workbook(train_file)
    sheet = wb['Sheet1']
    for row in sheet.iter_rows(min_row=2):  # 헤더는 불러오지 않음
        # 데이터 저장
        insert_data(db, row)

    wb.close()

except Exception as e:
    print(e)

finally:
    if db is not None:
        db.close()

안녕하세요 저장
반가워요 저장
{B_FOOD} 주문할게요 저장
{B_FOOD} 주문할게요 저장
{B_DT} 예약 저장
None 저장


### OpenPyXL 모듈을 이용해 엑셀 파일을 DB 데이터에 저장
insert 명령어를 사용하여 DB에 저장 <br>
select를 이용하여 결과 확인